In [30]:
from dataclasses import dataclass
@dataclass
class User:
    rank = -8
    progress = 0
    def inc_progress(self, p):
        if p == 0 or p not in range(-8, 9):
            raise Exception('out of range')
        
        dr = p-self.rank
        if p*self.rank < 0: dr -= p//abs(p)
        if dr < -1: return

        if dr == -1: dp = 1
        if dr == 0: dp = 3
        if dr > 0: dp = 10 * dr ** 2
        self.progress += dp
        
        while self.rank < 8 and self.progress >= 100:
            self.rank += 1
            if self.rank == 0: self.rank += 1
            self.progress -= 100
        if self.rank == 8: self.progress = 0


class test:
    def assert_equals(a,b):
        if a != b: print(a, 'not equal to', b)

user = User()
user.rank = 1

user.inc_progress(-1)
test.assert_equals(user.rank, 1)
test.assert_equals(user.progress, 1)
# test.assert_equals(user.rank, -8)
# test.assert_equals(user.progress, 0)
# user.inc_progress(-7)
# test.assert_equals(user.progress, 10)
# user.inc_progress(-5)
# test.assert_equals(user.progress, 0)
# test.assert_equals(user.rank, -7)


In [42]:
def recoverSecret(triplets):
    s = ''
    letters = set(c for l in triplets for c in l)
    while True:
        try: l = next(iter(letters))
        except: return s

        for t in triplets:
            if l in t and t.index(l) > 0: l = t[0]
        s += l
        letters.remove(l)
        triplets = [[c for c in t if c != l] for t in triplets]
        

secret = "whatisup"
triplets = [
  ['t','u','p'],
  ['w','h','i'],
  ['t','s','u'],
  ['a','t','s'],
  ['h','a','p'],
  ['t','i','s'],
  ['w','h','s']
]

test.assert_equals(recoverSecret(triplets), secret)


In [70]:
import numpy as np
from math import sqrt
class Sudoku(object):
  def __init__(self, data):
    self.d = data
  def is_valid(self):
    if set(type(c) for l in self.d for c in l) != {int}: return False

    try: d = np.array(self.d, dtype=np.uint)
    except: return False

    size = len(d)
    block_s = int(sqrt(size))
    digits = set(range(1, size+1))
    for l in d:
      if set(l) != digits: return False
    for c in d.T:
      if set(l) != digits: return False
    for i in range(size):
      x, y = (i%block_s)*block_s, (i//block_s)*block_s
      if set(d[x:x+block_s, y:y+block_s].flatten()) != digits: return False
    return True

# Valid Sudoku
goodSudoku1 = Sudoku([
  [7,8,4, 1,5,9, 3,2,6],
  [5,3,9, 6,7,2, 8,4,1],
  [6,1,2, 4,3,8, 7,5,9],

  [9,2,8, 7,1,5, 4,6,3],
  [3,5,7, 8,4,6, 1,9,2],
  [4,6,1, 9,2,3, 5,8,7],
  
  [8,7,6, 3,9,4, 2,1,5],
  [2,4,3, 5,6,1, 9,7,8],
  [1,9,5, 2,8,7, 6,3,4]
])

goodSudoku2 = Sudoku([
  [1,4, 2,3],
  [3,2, 4,1],

  [4,1, 3,2],
  [2,3, 1,4]
])

# Invalid Sudoku
badSudoku1 = Sudoku([
  [0,2,3, 4,5,6, 7,8,9],
  [1,2,3, 4,5,6, 7,8,9],
  [1,2,3, 4,5,6, 7,8,9],
  
  [1,2,3, 4,5,6, 7,8,9],
  [1,2,3, 4,5,6, 7,8,9],
  [1,2,3, 4,5,6, 7,8,9],
  
  [1,2,3, 4,5,6, 7,8,9],
  [1,2,3, 4,5,6, 7,8,9],
  [1,2,3, 4,5,6, 7,8,9]
])

badSudoku2 = Sudoku([
  [1,2,3,4,5],
  [1,2,3,4],
  [1,2,3,4],  
  [1]
])

class test:
    def assert_equals(a,b,r=''):
        if a != b: print(a, 'not equal to', b)

test.assert_equals(goodSudoku1.is_valid(), True, 'Testing valid 9x9')
test.assert_equals(goodSudoku2.is_valid(), True, 'Testing valid 4x4')

test.assert_equals(badSudoku1.is_valid(), False, 'Values in wrong order')
test.assert_equals(badSudoku2.is_valid(), False, '4x5 (invalid dimension)')

In [82]:
import re

OPS = [op for op in '+-*/']
def tokenize(e):
    e = e.replace(' ', '')
    token_specification = [
        ('NUMBER', r'\d+(\.\d*)?'),
        ('OP', r'[+\-\*/]'),
        ('LPAR', r'\('),
        ('RPAR', r'\)'),
        ('X', r'.'),
    ]
    tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    for mo in re.finditer(tok_regex, e):
        kind, value = mo.lastgroup, mo.group()
        if kind == 'NUMBER':
            value = float(value) if '.' in value else int(value)
        elif kind == 'X':
            raise RuntimeError(f'Unexpected value at {mo.start()}')
        yield value

def replace_op(at, tokens):
    a, op, b = tokens[at-1:at+2]
    if op == '+': r = a + b
    if op == '-': r = a - b
    if op == '*': r = a * b
    if op == '/': r = a / b
    if r == r//1: r = int(r)
    return tokens[:at-1] + [r] + tokens[at+2:]

def calc_tokens(tokens):
    while '(' in tokens:
        for i, t in enumerate(tokens):
            if t == '(': l = i
            if t == ')': r = i; break
        tokens = tokens[:l] + [calc_tokens(tokens[l+1:r])] + tokens[r+1:]

    i = 0
    while i < len(tokens):
        if tokens[i] == '-' and (i==0 or tokens[i-1] in OPS):
            tokens = tokens[:i] + [-tokens[i+1]] + tokens[i+2:]
        i+=1
    
    while '*' in tokens or '/' in tokens:
        for i, t in enumerate(tokens):
            if t in ['*', '/']: break
        tokens = replace_op(i, tokens)
    
    while len(tokens) > 1: tokens = replace_op(1, tokens)
    return tokens[0]

def calc(expr):
    return calc_tokens([t for t in tokenize(expr)])


# d = '7 * (5 + 6 / 3 - -5)'
# d = '3 -(-1)'

# print(d, '=', calc(d))


from cw_test import *
cases = (
    ("1 + 1", 2),
    ("8/16", 0.5),
    ("3 -(-1)", 4),
    ("2 + -2", 0),
    ("10- 2- -5", 13),
    ("(((10)))", 10),
    ("3 * 5", 15),
    ("-7 * -(6 / 3)", 14)
)

for x, y in cases:
    test.assert_equals(calc(x), y)


In [82]:
from dataclasses import field
from queue import PriorityQueue
import numpy as np
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, field

DIRS = {'L':(-1,0),'R':(1,0),'U':(0,-1),'D':(0,1)}
Pic = np.ndarray

def loopover(mixed_up_board: List[List[str]], solved_board: List[List[str]]) -> Optional[List[str]]:
    target_idx = {s: i for i, s in enumerate(np.array(solved_board).flat)}
    a = np.array([[target_idx[c] for c in l] for l in mixed_up_board])
    H, W = a.shape
    moves = [f'{d}{i}' for d in 'LR' for i in range(H)] + [f'{d}{i}' for d in 'UD' for i in range(W)]
    
    @dataclass(order=True)
    class Step:
        cost: int
        board: Pic = field(compare=False)
        steps: str = field(compare=False)
    
    flat_signature = lambda flat_bo: ''.join(map(str, flat_bo))
    target_signature = flat_signature(range(0, W*H))
    diff = lambda bo: sum([(r - (tile // W))**2 + (c - (tile % W))**2 for (r, c), tile in np.ndenumerate(bo)])
    
    # A*
    winning_steps = ''
    visited = set()
    q: PriorityQueue[Step] = PriorityQueue()

    root = Step(diff(a), a, '')
    if root.cost == 0: return []

    i = 0
    q.put(root)
    visited.add(flat_signature(a.flat))
    while not q.empty():
        i += 1
        if i % 1000 == 0: print(i)

        crt = q.get()

        for m in moves:
            # Do the move
            next_bo = crt.board.copy()
            move(next_bo, m, H, W)

            next_sig = flat_signature(next_bo)
            if next_sig in visited: continue

            # Diff to target pic
            next_diff = diff(next_bo)
            next_steps = crt.steps + m
            if next_diff == 0:
                winning_steps = next_steps
                q = PriorityQueue()
                break
            
            # Next step
            q.put(Step(
                len(next_steps) // 2 + next_diff,
                next_bo,
                next_steps
            ))
            visited.add(next_sig)

    res = [winning_steps[i:i+2] for i in range(0, len(winning_steps), 2)]
    print(res)

    return res

def move(a, m, H, W):
    ds, ns = m; n = int(ns)
    if ds == 'L': a[n] = np.concatenate((a[n,1:W], [a[n,0]]))
    if ds == 'R': a[n] = np.concatenate(([a[n,W-1]], a[n,:W-1]))
    if ds == 'U': a[:,n] = np.concatenate((a[1:H,n], [a[0,n]]))
    if ds == 'D': a[:,n] = np.concatenate(([a[H-1,n]], a[:H-1,n]))

def check(a, b, moves):
    a,b = map(np.array, [a,b])
    H, W = a.shape
    for m in moves: move(a, m, H, W)
    return np.array_equal(a, b)


a = np.array(range(16)).reshape(4,4)
H, W = a.shape
move(a, 'D2', H, W)
b = np.array(range(16)).reshape(4,4)

from cw_test import test
def run_test(start, end, unsolvable):
    def board(str):
        return [list(row) for row in str.split('\n')]
    # print(board(start), board(end))
    moves = loopover(board(start), board(end))
    if unsolvable:
        test.assert_equals(moves, None, 'Unsolvable configuration')
    else:
        test.assert_equals(check(board(start), board(end), moves), True)

# run_test('12\n34', '12\n34',  False)
# run_test('42\n31', '12\n34', False)
# run_test('425\n631', '123\n456', False)
run_test('ACDBE\nFGHIJ\nKLMNO\nPQRST', 'ABCDE\nFGHIJ\nKLMNO\nPQRST', False)
# run_test('ACDBE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', 'ABCDE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', False)
# run_test('ABCDE\nKGHIJ\nPLMNO\nFQRST\nUVWXY', 'ABCDE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', False)
# run_test('CWMFJ\nORDBA\nNKGLY\nPHSVE\nXTQUI', 'ABCDE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', False)
# run_test('WCMDJ\nORFBA\nKNGLY\nPHVSE\nTXQUI', 'ABCDE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', True)
# run_test('WCMDJ0\nORFBA1\nKNGLY2\nPHVSE3\nTXQUI4\nZ56789', 'ABCDEF\nGHIJKL\nMNOPQR\nSTUVWX\nYZ0123\n456789', False)

# TODO replace numpy with bitwise row/col shift

1000
2000
3000


KeyboardInterrupt: 

In [9]:
import sys
import math
from typing import Dict, Tuple
from queue import PriorityQueue
from dataclasses import dataclass, field

def log(*args):
    print(*args, file=sys.stderr, flush=True)

def draw(p: Dict):
    for i in range(3):
        log(' '.join([str(p[i, j]).ljust(2) for j in range(4)]))

zero_at = 0, 0
picture = {}
for row in range(3):
    for col, tile in enumerate(input().split()):
        t = int(tile)
        picture[(row, col)] = t
        if t == 0: zero_at = (row, col)

# draw(picture)
@dataclass(order=True)
class Step:
    cost: int
    picture: Dict = field(compare=False)
    zero_at: Tuple = field(compare=False)
    steps: str = field(compare=False)
dirs = {'v':(-1,0),'^':(1,0),'>':(0,-1),'<':(0,1)}

target_signature = ''.join(map(str, range(12)))
signature = lambda pic: ''.join(map(str, pic.values()))
diff = lambda pic: sum([abs(loc[0] - (tile // 4))**2 + abs(loc[1] - (tile % 4))**2 for loc, tile in pic.items()])

# A*
winning_steps = ''
visited = set()
q: PriorityQueue[Step] = PriorityQueue()

root = Step(diff(picture), picture, zero_at, '')
q.put(root)
visited.add(signature(picture))
while not q.empty():
    crt = q.get()

    zr, zc = crt.zero_at
    for d in dirs:
        dr, dc = dirs[d]
        moved_r, moved_c = zr + dr, zc + dc
        if not 0 <= moved_r < 3 or not 0 <= moved_c < 4:
            continue

        # Do the move
        next_zero_at = moved_r, moved_c
        next_pic = crt.picture.copy()
        next_pic[crt.zero_at] = next_pic[next_zero_at]
        next_pic[next_zero_at] = 0

        next_sig = signature(next_pic)
        if next_sig in visited:
            continue

        # Diff to target pic
        next_diff = diff(next_pic)
        next_steps = crt.steps + d
        if next_diff == 0:
            winning_steps = next_steps
            q = PriorityQueue()
            break
        elif len(next_steps) >= 49:
            continue

        # Next step
        q.put(Step(
            len(next_steps) + next_diff,
            next_pic,
            next_zero_at,
            next_steps
        ))
        visited.add(next_sig)

for d in winning_steps:
    zr, zc = zero_at
    dr, dc = dirs[d]
    zero_at = zr + dr, zc + dc
    print(f"{zero_at[0]} {zero_at[1]}")


4

In [114]:
# %%heat
from dataclasses import field
from queue import PriorityQueue
import numpy as np
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, field
from itertools import takewhile

cost_history = []

flatten = lambda a: [c for l in a for c in l]
DIRS = {'L':(-1,0),'R':(1,0),'U':(0,-1),'D':(0,1)}
Board = List[int]

def loopover(mixed_up_board: List[List[str]], solved_board: List[List[str]]) -> Optional[List[str]]:
    target_idx = {s: i for i, s in enumerate(flatten(solved_board))}
    a = [target_idx[c] for c in flatten(mixed_up_board)]
    H, W = len(mixed_up_board), len(mixed_up_board[0])
    moves = [f'{d}{i}' for d in 'LR' for i in range(H)] + [f'{d}{i}' for d in 'UD' for i in range(W)]
    lines, cols = [set(range(W*l, W*(l+1))) for l in range(H)], [set(range(c, W*H, W)) for c in range(W)]
    
    @dataclass(order=True)
    class Step:
        cost: int
        board: Board = field(compare=False)
        steps: str = field(compare=False)

    
    signature = lambda bo: ' '.join(map(str, bo))
    target_sig = signature(range(0, W*H))

    diff = lambda bo: sum([abs((i//W) - (tile//W)) + abs((i%W) - (tile%W)) for i, tile in enumerate(bo)]) # 0.9s solving time
    # diff = lambda bo: sum([min(dr%H, -dr%H) + min(dc%W, -dc%W) for bi, ai in enumerate(bo) if (dr:=(bi//W) - (ai//W), dc:=(bi%W) - (ai%W))])
    # diff = lambda bo: 1
    # row_diff ^ 2 + col_diff ^ 2
    # abs(row_diff) + abs(col_diff)
    # daca se misca toata lina / coloana o data: sa masoare cat de diferite sunt liniile coloanele
    # e.g. numarul de elemente lipsa din linie / coloana
    
    # diff = lambda bo: sum([len(set(bo[W*l : W*(l+1)]) - lines[l]) for l in range(H)]) + \
    #         sum([len(set(bo[c : W*H : W]) - cols[c]) for c in range(W)]) # 3.2s solving time
    # diff = lambda bo: sum([a!=b for a,b in zip(bo, range(W*H))])**2

    board_positions = {i for i in range(W*H)}
    progressive_squares=[0]
    while exp := {sq+d for sq in progressive_squares for d in (1, W, W+1)}\
        .difference(set(progressive_squares)).intersection(board_positions): progressive_squares += list(exp)

    # diff = lambda bo: W*H-len(list(takewhile(lambda i: bo[i]==i, progressive_squares)))
    if signature(a) == target_sig: return []
    
    i = 0

    # A*
    winning_steps = ''
    visited = set()
    q: PriorityQueue[Step] = PriorityQueue()
    q.put(Step(diff(a), a, ''))
    visited.add(signature(a))
    while not q.empty():
        crt = q.get()

        i += 1
        # if i % 100 == 0: 
        cost_history.append(crt.cost)
        # if i > 200000:
        #     print(len(visited))
        #     break
        
        for m in moves:
            # Do the move
            next_bo = crt.board.copy()
            move(next_bo, m, W, H)

            # Stop if place already visited
            next_sig = signature(next_bo)
            if next_sig in visited: continue
            
            # Diff to target board
            next_diff = diff(next_bo)
            next_steps = crt.steps + m
            if next_diff == 0:
                print(next_bo)
                winning_steps = next_steps
                q = PriorityQueue()
                break
            
            # Next step
            q.put(Step(
                len(next_steps) // 2 + next_diff,
                next_bo,
                next_steps
            ))
            visited.add(next_sig)

    res = [winning_steps[i:i+2] for i in range(0, len(winning_steps), 2)]
    return res

def move(a: Board, m, W, H):
    ds, ns = m; n = int(ns)

    # Ln = range(W*n, W*(n+1)-1, 1)
    # Rn = range(W*(n+1)-1, W*n-1, -1)
    # Un = range(n, W*(H-1)+n, W)
    # Dn = range(W*(H-1)+n, n, -W)
    if ds in 'LR': start, end, step = W*n, W*(n+1)-1, 1
    if ds in 'UD': start, end, step = n, W*(H-1)+n, W
    if ds in 'RD': start, end, step = end, start, -step
    for i in range(start, end, step): a[i], a[i+step] = a[i+step], a[i]

def pretty(a):
    for i in range(H): print(a[W*i:W*(i+1)])

def check(a_init: List[List[str]], b_init: List[List[str]], moves: List[str]):
    H, W = len(a_init), len(a_init[0])
    a, b = map(flatten, [a_init, b_init])

    # print(pretty(a))
    for m in moves: move(a, m, W, H)#; print(pretty(a))
    
    return sum([ai!=bi for ai, bi in zip(a, b)]) == 0

from cw_test import test
def run_test(start, end, unsolvable):
    def board(str): return [list(row) for row in str.split('\n')]
    moves = loopover(board(start), board(end))
    print('moves', moves)
    if unsolvable:
        test.assert_equals(moves, None, 'Unsolvable configuration')
    else:
        test.assert_equals(check(board(start), board(end), moves), True)

# run_test('12\n34', '12\n34',  False)
# run_test('42\n31', '12\n34', False)
run_test('425\n631', '123\n456', False)
# run_test('012\n345\n678', '012\n345\n678', False)
# run_test('120\n345\n678', '012\n345\n678', False)
# run_test('210\n345\n678', '012\n345\n678', False)
# run_test('ACDBE\nFGHIJ\nKLMNO\nPQRST', 'ABCDE\nFGHIJ\nKLMNO\nPQRST', False)
# run_test('ACDBE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', 'ABCDE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', False)
# run_test('ABCDE\nKGHIJ\nPLMNO\nFQRST\nUVWXY', 'ABCDE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', False)
# run_test('CWMFJ\nORDBA\nNKGLY\nPHSVE\nXTQUI', 'ABCDE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', False)
# run_test('WCMDJ\nORFBA\nKNGLY\nPHVSE\nTXQUI', 'ABCDE\nFGHIJ\nKLMNO\nPQRST\nUVWXY', True)
# run_test('WCMDJ0\nORFBA1\nKNGLY2\nPHVSE3\nTXQUI4\nZ56789', 'ABCDEF\nGHIJKL\nMNOPQR\nSTUVWX\nYZ0123\n456789', False)
len(cost_history)

[0, 1, 2, 3, 4, 5]
moves ['U2', 'R1', 'U2', 'R1', 'U0']


33

In [119]:
import itertools

W=H=4
a=list(range(W*H))
move(a, 'R1', W, H)
# move(a, 'D2', W, H)
# move(a, 'L1', W, H)

t_format = lambda a: '\n'.join([' '.join(str(c).ljust(len(str(W*H))) for c in a[W*i:W*(i+1)]) for i in range(H)])
print(t_format(a))
board = lambda a: [a[W*i:W*(i+1)] for i in range(H)]
# print('moves', loopover(board(a), board(range(W*H))))

# sum([
#     (
#         min(abs((bi//W) - (ai//W)), (bi//W) - (ai//W)) +
#         min(abs((bi%W) - (ai%W)), ((bi%W) - (ai%W)))
#     )
#     for bi, ai in enumerate(a)
# ])

board_positions = {i for i in range(W*H)}
progressive_squares=[0]
while exp := {sq+d for sq in progressive_squares for d in (1, W, W+1)}\
    .difference(set(progressive_squares)).intersection(board_positions): progressive_squares += list(exp)

W*H-len(list(itertools.takewhile(lambda i: a[i]==i, progressive_squares)))



0  1  2  3 
7  4  5  6 
8  9  10 11
12 13 14 15


14

In [85]:
import math

W,H=4,4
a=range(W*H)
t_format = lambda a: '\n'.join([' '.join(str(c).ljust(len(str(W*H))) for c in a[W*i:W*(i+1)]) for i in range(H)])
print(t_format(a))
am=[a[W*i:W*(i+1)] for i in range(H)]
ai=[(r,c) for c in range(H) for r in range(W)]

available_squares = {i for i in range(W*H)}
diff_squares=[0]
while exp := {sq+d for sq in diff_squares for d in (1, W, W+1)}\
    .difference(set(diff_squares)).intersection(available_squares): diff_squares += list(exp)

diff_squares


0  1  2  3 
4  5  6  7 
8  9  10 11
12 13 14 15
{1, 4, 5}
{2, 6, 8, 9, 10}
{3, 7, 11, 12, 13, 14, 15}


[0, 1, 4, 5, 2, 6, 8, 9, 10, 3, 7, 11, 12, 13, 14, 15]